In [3]:
#расширяем экран
from IPython.display import display, HTML
display(HTML('<style>.container { width:100% !important; }</style>'))

In [2]:
import requests
import pandas as pd
import pendulum
import time
import smtplib
from datetime import datetime, timedelta
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.providers.telegram.operators.telegram import TelegramOperator
#from db_resources import get_postgres_connection_string
from sqlalchemy import create_engine, text
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.header import Header
from email import encoders
from pretty_html_table import build_table
import numpy as np

In [3]:
connection_ = 'postgresql(...)'

In [4]:
    # Выполняем подключение к БД
    engine2 = create_engine(connection_)
    engine3 = create_engine(connection_) 
    #

In [108]:
    # выгружаем список действующих филиалов 
    # используется своя таблица с набором данных. 
    filial_list = pd.read_sql(''' 
        select
			firmy."Ссылка" as filial_guid,
			firmy."Наименование" as "Филиал",
			territorii."Наименование" as "РРС"
			from
			"_"."_" as firmy
            left join (
                select
                "Ссылка",
                "Наименование"
                from "_"."S.Territorii"
                where
                "Ссылка" in ('47449ffc-7ca8-11e5-a729-00155d03361b' , '794dd148-409b-11e1-8064-001517c526f0' , '312873ac-708d-11e5-9610-00155d03361b',
        		'4979cede-2660-11e7-ab3b-00155d03330d', 'b80e971f-f028-11e2-80d0-00155d030b1f', 'b10b3c28-f137-11e9-a20d-00155d03332b', '894248bc-8246-11e2-92a8-00155d030b1f') 
                ) as territorii
        	on
            firmy."ПриписанК_Территории" = territorii."Ссылка"
			where
			firmy."ПриписанК_Территории" in ( 
                '47449ffc-7ca8-11e5-a729-00155d03361b' /* РРС ДВ "Сахалин" */ , 
                '794dd148-409b-11e1-8064-001517c526f0' /* РРС ДВ "Благовещенск" */ ,
                '312873ac-708d-11e5-9610-00155d03361b' /* РРС ДВ "Хабаровск" */ ,
                '4979cede-2660-11e7-ab3b-00155d03330d' /* РРС ДВ "Камчатка" */, 
                'b80e971f-f028-11e2-80d0-00155d030b1f' /* РРС ДВ "Приморье" */, 
                'b10b3c28-f137-11e9-a20d-00155d03332b' /* РРС ДВ "Комсомольск" */ ,
                '894248bc-8246-11e2-92a8-00155d030b1f' /* РРС ДВ "Владивосток" */)
			and firmy."ПометкаУдаления" = 0 
			and firmy."ВидФилиала" in ('d460d799-3e63-44ae-b180-29fa0f12d869')
    ''', engine2)
    #

In [109]:
filial_list

,filial_guid,Филиал,РРС
0,89faec91-6d7f-11dc-a590-00151716f9f5,Комсомольск Ленина МБТ,"РРС ДВ ""Комсомольск"""
1,174c2013-8f1f-4db4-b8a9-dafedc66feac,Камень-Рыболов Кирова КБТ,"РРС ДВ ""Приморье"""
2,3e2710fb-8ad1-11eb-a21b-00155d214d08,Вилючинск ТЦ Вилюй КБТ,"РРС ДВ ""Камчатка"""
3,45406e49-f1d1-4fc8-82a7-d702a87d0bfb,Солнечный Ленина КБТ,"РРС ДВ ""Комсомольск"""
4,718dcf7b-aacc-4d93-9182-facfdbd4ec61,Раздольное Лазо КБТ,"РРС ДВ ""Владивосток"""
...,...,...,...
117,885da112-7206-4802-a0ce-3edee4cb573b,Хаб ТЦ Северный SMART,"РРС ДВ ""Хабаровск"""
118,b342a254-53e7-44ab-92e3-57f6aff7a307,Углегорск Победы КБТ,"РРС ДВ ""Сахалин"""
119,ff411c94-4ae6-471f-b1e1-a4ec36734412,Серышево Универмаг КБТ,"РРС ДВ ""Благовещенск"""
120,da18a110-7605-4082-980d-c64991cf12b6,Райчихинск ТЦ Сити КБТ,"РРС ДВ ""Благовещенск"""


In [104]:
    # выгружаем список email и филиал сотрудника для объединения с результатом запроса API 
    # используется своя таблица с набором данных. 
    email_list = pd.read_sql('''
SELECT distinct 
	p."_IDRRef" AS UserGUID,
    --p."_Marked" AS "ПометкаУдаления",
    --p."_Code" AS "Код",
    p."_Description" AS Пользователь,
    --p."_Fld2457RRef" AS "ЯвляетсяСотрудникомФилиала",
    --p."_Fld205RRef" AS "ФизЛицо",
    --p."_Fld206RRef" AS "ОсновнаяФирма",
    --p."_Fld207RRef" AS "ПроводитьПо",
    --p."_Fld15263" AS "ДатаРождения",
    p."_Fld15459" AS email,
    firmy."Ссылка" as ФилиалGUID,
    firmy."Наименование" as Филиал,
    p."_Fld35000RRef" AS ДолжностьGUID,
    (case 
    	when p."_Fld35000RRef" = '4c10d85a-be27-45ed-b366-19c090ea0e7a' then 'Управляющий маг. (РЕГ)'
    	when p."_Fld35000RRef" = '22747784-a29e-4ab3-bc6d-0fc91535327e' then 'Управляющий маг. средней категории (РЕГ)'
    	when p."_Fld35000RRef" = 'c39be1e8-0864-4e50-a2fc-3e3b13198834' then 'Управляющий маг. высшей категории (РЕГ)'
    	when p."_Fld35000RRef" = '41d6e060-8c81-4487-a83a-2f4a2b45e885' then 'Зам. управляющего магазина (РЕГ)'
    end) AS dolzhnost
FROM ""."" as p
	 join (
	 		select
			firmy."Ссылка",
			firmy."Наименование"
			--firmy."ПриписанК_Территории"
			from
			""."" as firmy
			where
			firmy."ПриписанК_Территории" in ( 
			'47449ffc-7ca8-11e5-a729-00155d03361b' /* РРС ДВ "Сахалин" */ , 
			'794dd148-409b-11e1-8064-001517c526f0' /* РРС ДВ "Благовещенск" */ ,
			'312873ac-708d-11e5-9610-00155d03361b' /* РРС ДВ "Хабаровск" */ ,
			'4979cede-2660-11e7-ab3b-00155d03330d' /* РРС ДВ "Камчатка" */, 
			'b80e971f-f028-11e2-80d0-00155d030b1f' /* РРС ДВ "Приморье" */, 
			'b10b3c28-f137-11e9-a20d-00155d03332b' /* РРС ДВ "Комсомольск" */ ,
			'894248bc-8246-11e2-92a8-00155d030b1f' /* РРС ДВ "Владивосток" */)
			and firmy."ПометкаУдаления" = 0 
			and firmy."ВидФилиала" in ('d460d799-3e63-44ae-b180-29fa0f12d869')
	 		) 
	 		as firmy on p."_Fld2457RRef"=firmy."Ссылка"
where p."_Fld35000RRef" in ('4c10d85a-be27-45ed-b366-19c090ea0e7a', '22747784-a29e-4ab3-bc6d-0fc91535327e', '41d6e060-8c81-4487-a83a-2f4a2b45e885', 'c39be1e8-0864-4e50-a2fc-3e3b13198834') 
	--and date(p."$_inner_ts")=date(now())
	and p."_Marked" = '0'
    ''', engine2)
    #

In [1]:
email_list

NameError: name 'email_list' is not defined

In [194]:
    # выгружаем список email и филиал сотрудника для объединения с результатом запроса API 
    # используется своя таблица с набором данных. 
    vidacha_iz= pd.read_sql('''
with firmy as (select
			firmy."Ссылка",
			firmy."Наименование" ,
			firmy."ПриписанК_Территории"
			from
			"_"."_" as firmy
			where
			firmy."ПриписанК_Территории" in ( 
                '47449ffc-7ca8-11e5-a729-00155d03361b' /* РРС ДВ "Сахалин" */ , 
                '794dd148-409b-11e1-8064-001517c526f0' /* РРС ДВ "Благовещенск" */ ,
                '312873ac-708d-11e5-9610-00155d03361b' /* РРС ДВ "Хабаровск" */ ,
                '4979cede-2660-11e7-ab3b-00155d03330d' /* РРС ДВ "Камчатка" */, 
                'b80e971f-f028-11e2-80d0-00155d030b1f' /* РРС ДВ "Приморье" */, 
                'b10b3c28-f137-11e9-a20d-00155d03332b' /* РРС ДВ "Комсомольск" */ ,
                '894248bc-8246-11e2-92a8-00155d030b1f' /* РРС ДВ "Владивосток" */)
            ),
	SPZD as (select
			spzd_."ID",
			spzd_."Автор",
			spzd_."Событие",
			spzd_."Период",
			spzd_."Филиал",
			spz_."ТипЗадания",
			--spz_doc."Документ",
			spz_."Документ",
			spzd_."ИсточникСобытия"
			from "_"."_" as spzd_
			inner join "_"."_" as spz_ on
			spzd_."ID" = spz_."ID"
			/*left join "_"."_" as spz_doc on
			spzd_."ID" = spz_doc."ID"*/
			where (
					spzd_."Период" between ( DATE(now()) - interval '8 days') and ( DATE(now()) - interval '1 days') /* DATE(:date2) */
					)
					and spzd_."Событие" in (1, 101, 105)
					and spz_."ТипЗадания" in ('d34ca20c-3ab4-11ec-8f1b-00155d8ed20b') 
					and spzd_."Филиал" in (select firmy."Ссылка" from firmy)
			), 
	date_ as (select 
			Date(date_trunc('day', SPZD."Период")) as day_trunc,
			SPZD."Период",
			SPZD."ID"
			from SPZD
			),
	_1_ as (select
			SPZD."ID",
			SPZD."Событие",
			SPZD."Автор",
			SPZD."Филиал",
			SPZD."ТипЗадания",
			SPZD."Документ",
			SPZD."ИсточникСобытия" as ИС_1,
			max(SPZD."Период") as date_1_ 
			from SPZD 
			where SPZD."Событие"=1 
			group by 1,2,3,4,5,6,7
			), 
	_101_ as (select
			SPZD."ID",
			SPZD."Событие",
			SPZD."ИсточникСобытия" as ИС_101,
			max(SPZD."Период") as date_101_ 
			from SPZD 
			where SPZD."Событие"=101 
			group by 1,2,3
			), 
	_105_ as (select distinct
			SPZD."ID",
			SPZD."Событие",
			SPZD."Автор",
			SPZD."Филиал",
			SPZD."ТипЗадания",
			SPZD."Документ",
			SPZD."ИсточникСобытия",
			max(SPZD."Период") as date_105_
			from SPZD
			where SPZD."Событие"=105
			group by 1,2,3,4,5,6,7
			),
	total as (select distinct 
			_105_."Филиал" ,
			firmy."ПриписанК_Территории",
			_105_."Документ",
			_105_."ТипЗадания",
			_105_."ИсточникСобытия",
			_1_.ИС_1,
			date_.day_trunc,
			_105_."ID",
			_1_.date_1_,
			_101_.date_101_,
			_105_.date_105_,
			(_101_.date_101_-_1_.date_1_) as _1_101_date_diff_time,
			EXTRACT( EPOCH from _101_.date_101_-_1_.date_1_) as _1_101_date_diff_time_sec,
			(_105_.date_105_-_101_.date_101_) as _101_105_date_diff_time,
			EXTRACT( EPOCH from _105_.date_105_-_101_.date_101_) as _101_105_date_diff_time_sec,
			_105_."Автор"  
			from _105_
			left join _1_ on _105_."ID"=_1_."ID"
			left join date_ on _105_.date_105_=date_."Период"  
			left join firmy on _105_."Филиал"=firmy."Ссылка"
			left join _101_ on _105_."ID"=_101_."ID"
			where (
				  	(extract(EPOCH from _105_.date_105_-_101_.date_101_) between 10 and 7200
								and _105_."ТипЗадания" 
								in ('d34ca20c-3ab4-11ec-8f1b-00155d8ed20b') 
				  	) 
				  and
				  	(extract(EPOCH from _101_.date_101_-_1_.date_1_) between 10 and 7200
								and _105_."ТипЗадания" 
								in ('d34ca20c-3ab4-11ec-8f1b-00155d8ed20b') 		 
				  	) 
				  )
				  and _105_.date_105_  > _101_.date_101_
				  and _101_.date_101_ > _1_.date_1_
				  --and _105_."ТипЗадания" in ('35041486-c85d-11ed-90b8-00155d8ed20b')
                )
			--,
		--total as (select * from _1_ full outer join total_101_105 on total_101_105."ID"=_1_."ID")
--select * from total;
select distinct 
	total.day_trunc as ДатаВыпПоручения,
	--territorii."Наименование" as РРС,
	total."Филиал" as ФилиалGUID,
	firmy."Наименование" as Филиал ,
	SPTZ."Наименование" as ТипЗадания,
	round(avg(total._101_105_date_diff_time_sec::numeric/60), 2) as ВремяВыполненияПорученияМинут,
	count(total."ID") as КоличествоПоручений
from total
left join (
    select
	Ссылка",
	"Наименование"
	from "_"."_"
	where
	"Ссылка" in ('47449ffc-7ca8-11e5-a729-00155d03361b' , '794dd148-409b-11e1-8064-001517c526f0' , '312873ac-708d-11e5-9610-00155d03361b',
    '4979cede-2660-11e7-ab3b-00155d03330d', 'b80e971f-f028-11e2-80d0-00155d030b1f', 'b10b3c28-f137-11e9-a20d-00155d03332b', '894248bc-8246-11e2-92a8-00155d030b1f') 
        ) as territorii
on total."ПриписанК_Территории" = territorii."Ссылка"
left join (
	select
	"Ссылка",
	"Наименование",
	"Код"
	from "_"."_"
	where
	"Ссылка" in ( 'd34ca20c-3ab4-11ec-8f1b-00155d8ed20b' , 'ca94fa7f-6a2d-11e8-9547-00155d03330d', 'e4a53c12-cebf-11e9-a20f-00155d039a15',
	'f68b5287-e576-11e9-a211-00155d039a15', '073f1f97-c3d1-11e9-a20c-00155d03332b', '19f82228-6a30-11e8-9547-00155d03330d',
	'0c31b207-257c-11ec-8efb-00155d8ed20b', '74a393f6-0bdc-11ed-8ff2-00155d8ed20b', 'eb7e7aaf-b6dc-11eb-a262-00155dd2ff18', '35041486-c85d-11ed-90b8-00155d8ed20b') 
		) as SPTZ
on total."ТипЗадания" = SPTZ."Ссылка"
left join 
	firmy
on total."Филиал" = firmy."Ссылка"
left join (
	select
	"_IDRRef",
	"_Description"
	from "_"."_"
        ) as author
on total."Автор" = author."_IDRRef"
	--where (total.day_trunc = '2023-04-06') AND (total."Документ" = '5d30f11a-9ba6-4202-b8a2-bba7d752b01c')
	--group by 16,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17
	group by 1,2,3,4
/*(ДатаВыпПоручения = '2023-04-06') AND (Документguid = '5d30f11a-9ba6-4202-b8a2-bba7d752b01c')*/
    ''', engine2)

In [197]:
vidacha_iz

,ДатаВыпПоручения,Филиалguid,Филиал,ТипЗадания,ВремяВыполненияПорученияМинут,КоличествоПоручений
0,2023-08-12,96031d96-fb8c-4fba-a6b1-74105ba97a71,Комсомольск на Комшоссе Гипер,Выдача интернет-заказа,9.65,10
1,2023-08-14,174c2013-8f1f-4db4-b8a9-dafedc66feac,Камень-Рыболов Кирова КБТ,Выдача интернет-заказа,0.62,1
2,2023-08-12,9661dfc0-dbca-4ee8-a44d-ebf1d47578c7,Большой Камень ТЦ Айсберг КБТ,Выдача интернет-заказа,6.62,17
3,2023-08-13,87d31da5-7e09-4ae8-96ae-b7e6d0c7312d,Холмск ТЦ Колизей Гипер,Выдача интернет-заказа,1.38,2
4,2023-08-12,69e420cf-a1d8-11db-a40b-00001a1933df,Владивосток Русская КБТ,Выдача интернет-заказа,2.30,15
...,...,...,...,...,...,...
545,2023-08-08,d14ee7b0-9335-4237-9de1-613b5f8e2b8f,Южно-Сахалинск Панорама Гипер,Выдача интернет-заказа,2.64,23
546,2023-08-11,d14ee7b0-9335-4237-9de1-613b5f8e2b8f,Южно-Сахалинск Панорама Гипер,Выдача интернет-заказа,2.31,35
547,2023-08-10,7e950138-37b6-424f-ad3a-da422dfdbe5f,П-Камчатский ТЦ Пирамида,Выдача интернет-заказа,0.32,1
548,2023-08-08,b330b6b0-5ac6-4599-a389-9389a02cb4cd,П-Камчатский ТЦ Галант Гипер,Выдача интернет-заказа,3.58,14


In [114]:
    # выгружаем список email и филиал сотрудника для объединения с результатом запроса API 
    # используется своя таблица с набором данных. 
    vidacha_iz_rrs= pd.read_sql(''' 
        ... 
    ''', engine2)

In [115]:
vidacha_iz_rrs['date_end'].max()

datetime.date(2023, 8, 15)

In [116]:
vidacha_iz_rrs

,РРС,Филиал,ТипЗадания,date_start,date_end,ВремяВыполненияПорученияМинут,КоличествоПоручений
0,"РРС ДВ ""Владивосток""",Владивосток ТЦ Первореченский SMART,Выдача интернет-заказа,2023-08-08,2023-08-15,0.78,16
1,"РРС ДВ ""Владивосток""",Находка ТЦ Мега Гипер,Выдача интернет-заказа,2023-08-08,2023-08-15,4.97,107
2,"РРС ДВ ""Сахалин""",Южно-Сахалинск Ленина ТП,Выдача интернет-заказа,2023-08-08,2023-08-15,2.47,92
3,"РРС ДВ ""Владивосток""",Артем ТЦ Конкорд,Выдача интернет-заказа,2023-08-08,2023-08-15,2.03,85
4,"РРС ДВ ""Комсомольск""",Совгавань на Пионерской Гипер,Выдача интернет-заказа,2023-08-08,2023-08-15,2.69,95
...,...,...,...,...,...,...,...
90,"РРС ДВ ""Приморье""",Арсеньев ТЦ Радуга Гипер,Выдача интернет-заказа,2023-08-08,2023-08-15,3.81,287
91,"РРС ДВ ""Владивосток""",Владивосток Снеговая падь МБТ,Выдача интернет-заказа,2023-08-08,2023-08-15,3.97,51
92,"РРС ДВ ""Владивосток""",Владивосток ТЦ Маяк КБТ,Выдача интернет-заказа,2023-08-08,2023-08-15,3.95,89
93,"РРС ДВ ""Владивосток""",Артем Пушкина КБТ,Выдача интернет-заказа,2023-08-08,2023-08-15,3.25,188


In [117]:
#klusters=pd.read_excel('кластеры_июль,_отношение_по_часам.xlsx',usecols='A:C')
klusters=pd.read_excel('кластеры_июль,_отношение_по_часам.xlsx', sheet_name='k', usecols='A:C')
klusters.columns = ['РРС', 'Филиал', 'Итоговый кластер']

kluster_rrs = pd.merge(klusters, vidacha_iz_rrs, how='outer', on=["Филиал", "РРС"])
kluster_rrs = pd.merge(kluster_rrs, filial_list, how='outer', on=["Филиал", "РРС"])
kluster_rrs = kluster_rrs.drop(['filial_guid'], axis=1)
date_start = str(kluster_rrs['date_start'][0])
date_end = str(kluster_rrs['date_end'][0])
values1 = {'Итоговый кластер':0,'ТипЗадания':'Выдача интернет-заказа', 'date_start':date_start, 'date_end':date_end}
kluster_rrs = kluster_rrs.fillna(value=values1)
kluster_rrs = kluster_rrs.fillna(0)
kluster_rrs['Итоговый кластер']=kluster_rrs['Итоговый кластер'].astype(int)
kluster_rrs['КоличествоПоручений']=kluster_rrs['КоличествоПоручений'].astype(int)

kluster_rrs=kluster_rrs.drop(kluster_rrs[kluster_rrs['Филиал'].str.contains('(ЗАКРЫТО)')].index)
kluster_rrs=kluster_rrs.drop(kluster_rrs[kluster_rrs['Филиал'].str.contains('ТП')].index)

C:\Users\Vasilev.IEh\AppData\Local\Temp\ipykernel_21172\730559768.py:16 UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.

In [118]:
kluster_rrs

,РРС,Филиал,Итоговый кластер,ТипЗадания,date_start,date_end,ВремяВыполненияПорученияМинут,КоличествоПоручений
0,"РРС ДВ ""Владивосток""",Владивосток ТЦ Первореченский SMART,3,Выдача интернет-заказа,2023-08-08,2023-08-15,0.78,16
1,"РРС ДВ ""Благовещенск""",Завитинск ТЦ Наш КБТ,3,Выдача интернет-заказа,2023-08-08,2023-08-15,0.00,0
2,"РРС ДВ ""Хабаровск""",Хаб ТЦ Воронежский КБТ,1,Выдача интернет-заказа,2023-08-08,2023-08-15,0.00,0
3,"РРС ДВ ""Камчатка""",Вилючинск Вилкова КБТ,3,Выдача интернет-заказа,2023-08-08,2023-08-15,1.69,4
4,"РРС ДВ ""Хабаровск""",Облучье ТЦ Статик КБТ,3,Выдача интернет-заказа,2023-08-08,2023-08-15,0.68,1
...,...,...,...,...,...,...,...,...
118,"РРС ДВ ""Сахалин""",Южно-Сахалинск Бумажная Гипер,0,Выдача интернет-заказа,2023-08-08,2023-08-15,2.79,128
119,"РРС ДВ ""Хабаровск""",Хаб ТЦ СВИФ КБТ,0,Выдача интернет-заказа,2023-08-08,2023-08-15,2.96,9
120,"РРС ДВ ""Хабаровск""",Вяземский на Орджоникидзе КБТ,0,Выдача интернет-заказа,2023-08-08,2023-08-15,0.00,0
121,"РРС ДВ ""Владивосток""",Интернет Магазин Артем,0,Выдача интернет-заказа,2023-08-08,2023-08-15,0.00,0


In [119]:
kluster_table=kluster_rrs.drop(['Филиал','date_start', 'date_end'], axis=1)
kluster_table['РРС']=kluster_table['Итоговый кластер']
kluster_table=kluster_table.drop(['Итоговый кластер'], axis=1)
kluster_table.columns = ['Итоговый кластер', 'Тип Задания','Выполнение поручения в минутах', 'Количество поручений']
kluster_table = kluster_table.groupby(['Итоговый кластер', 'Тип Задания'], as_index=False).agg(Выполнение_поручения_в_минутах=('Выполнение поручения в минутах','mean'), Количество_поручений=('Количество поручений','sum'))
kluster_table.columns= ['Итоговый кластер', 'Тип Задания', 'Выполнение поручения в минутах', 'Количество поручений']
kluster_table=kluster_table.round({'Итоговый кластер':0,'Выполнение поручения в минутах': 2, 'Количество поручений': 0})

In [120]:
kluster_table

,Итоговый кластер,Тип Задания,Выполнение поручения в минутах,Количество поручений
0,0,Выдача интернет-заказа,1.10,156
1,1,Выдача интернет-заказа,0.71,62
2,2,Выдача интернет-заказа,2.94,602
3,3,Выдача интернет-заказа,0.54,51
4,4,Выдача интернет-заказа,2.64,1071
5,5,Выдача интернет-заказа,1.01,41
6,6,Выдача интернет-заказа,2.60,1171
7,7,Выдача интернет-заказа,2.88,1561
8,8,Выдача интернет-заказа,3.34,629
9,10,Выдача интернет-заказа,1.15,165


In [7]:
test_directors={'USER_NAME1':'USER_GMAIL1','USER_NAME2':'USER_GMAIL2', 'USER_NAME3':'USER_GMAIL3'}
test_directors

{'USER_NAME1': 'USER_GMAIL1',
 'USER_NAME2': 'USER_GMAIL2',
 'USER_NAME3': 'USER_GMAIL3'}

In [8]:
rrs_integrators={'USER_NAME4':'USER_GMAIL4','USER_NAME5':'USER_GMAIL5', 'USER_NAME6':'USER_GMAIL6'}
rrs_integrators

{'USER_NAME4': 'USER_GMAIL4',
 'USER_NAME5': 'USER_GMAIL5',
 'USER_NAME6': 'USER_GMAIL6'}

In [95]:
# Достаем список емейлов для рассылки
email = email_list['email']
#
# Настройки почтового клиента для рассылки 
login = 'USER_LOGIN'
sender_email = 'USER_EMAIL'
password = 'USER_PASSWORD'
smtp_server = '_'
smtp_port = 587
#Список пользователь по умолчанию в копию письма
owners = 'USER_NAME1','USER_NAME2', 'USER_NAME3'
distribution_email = list(rrs_integrators.values())
#

In [128]:
kluster_rrs.drop(['date_start', 'date_end'], axis=1) # какие данные будут в экселе

,РРС,Филиал,Итоговый кластер,ТипЗадания,ВремяВыполненияПорученияМинут,КоличествоПоручений
0,"РРС ДВ ""Владивосток""",Владивосток ТЦ Первореченский SMART,3,Выдача интернет-заказа,0.78,16
1,"РРС ДВ ""Благовещенск""",Завитинск ТЦ Наш КБТ,3,Выдача интернет-заказа,0.00,0
2,"РРС ДВ ""Хабаровск""",Хаб ТЦ Воронежский КБТ,1,Выдача интернет-заказа,0.00,0
3,"РРС ДВ ""Камчатка""",Вилючинск Вилкова КБТ,3,Выдача интернет-заказа,1.69,4
4,"РРС ДВ ""Хабаровск""",Облучье ТЦ Статик КБТ,3,Выдача интернет-заказа,0.68,1
...,...,...,...,...,...,...
118,"РРС ДВ ""Сахалин""",Южно-Сахалинск Бумажная Гипер,0,Выдача интернет-заказа,2.79,128
119,"РРС ДВ ""Хабаровск""",Хаб ТЦ СВИФ КБТ,0,Выдача интернет-заказа,2.96,9
120,"РРС ДВ ""Хабаровск""",Вяземский на Орджоникидзе КБТ,0,Выдача интернет-заказа,0.00,0
121,"РРС ДВ ""Владивосток""",Интернет Магазин Артем,0,Выдача интернет-заказа,0.00,0


In [130]:
kluster_filename = f"kluster_rrs_{date_start}_{date_end}.xlsx"
kluster_rrs.columns = ['РРС', 'Филиал', 'Итоговый кластер', 'Тип Задания', 'date_start', 'date_end', 'Выполнение поручения в минутах', 'Количество поручений']
kluster_rrs.drop(['date_start', 'date_end'], axis=1).to_excel(kluster_filename, index=False)

In [11]:
#Основной цикл отправки - цикл по списку почт людей, кому нужно отправить. Отправляется интеграторам-они смотрят на всю ситуацию в дивизионе в целом, поэтому к ним придохят данные со всех магазинов за последнюю неделю

for chel_email in rrs_integrators.values():
#time.sleep(10)
    subject = 'Скорость выполнения выдачи интернет-заказа'
    rrs_body = '<html><body>'
    rrs_body += f'<h2>Скорость выполнения выдачи интернет-заказа в среднем за прошедшую неделю (c {date_start} по {date_end}):</h2>\n\n'
    rrs_table = build_table(kluster_table, 'orange_dark', font_size='11px', padding= '0px , 0px,  0px , 0px', text_align='center', width_dict=['310px', '310px', '310px', '310px'])
    rrs_body += f"{rrs_table}" 
    rrs_body += '<p><em>Письмо сформировано автоматически, отвечать не требуется</em><br /><em>С уважением, Отдел сопровождения проектов</em></p>'
    rrs_body += '</body></html>'
    rrs_email_text = MIMEMultipart()
    rrs_email_text.attach(MIMEText(rrs_body, 'html'))
    part = MIMEBase('application', "octet-stream")
    part.set_payload(open(f"kluster_rrs_{date_start}_{date_end}.xlsx", "rb").read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', f'attachment; filename={kluster_filename}')
    rrs_email_text.attach(part)
    rrs_email_text['From'] = sender_email
    rrs_email_text['To'] = 'USER_EMAIL' #", ".join(distribution_email)
    #rrs_email_text['CC'] = ", ".join(distribution_email)
    #rrs_email_text['CC'] = ','.join(list(owners))
    rrs_email_text['Subject'] = Header(subject, 'utf-8')
    ## Отправка письма
    time.sleep(3)
    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(login, password)
        server.sendmail(sender_email , distribution_email, rrs_email_text.as_string())
        print(f"Сообщение отправлено на {distribution_email}")
        time.sleep(3)
        server.quit()
    except Exception as e:
        print(f"Ошибка с отправкой сообщения на {'USER_EMAIL'}: {str(e)}")
    break

NameError: name 'date_start' is not defined

In [14]:
owners

NameError: name 'owners' is not defined

In [15]:
distribution_email

NameError: name 'distribution_email' is not defined

In [148]:
kluster_rrs

,РРС,Филиал,Итоговый кластер,Тип Задания,date_start,date_end,Выполнение поручения в минутах,Количество поручений
0,"РРС ДВ ""Владивосток""",Владивосток ТЦ Первореченский SMART,3,Выдача интернет-заказа,2023-08-08,2023-08-15,0.78,16
1,"РРС ДВ ""Благовещенск""",Завитинск ТЦ Наш КБТ,3,Выдача интернет-заказа,2023-08-08,2023-08-15,0.00,0
2,"РРС ДВ ""Хабаровск""",Хаб ТЦ Воронежский КБТ,1,Выдача интернет-заказа,2023-08-08,2023-08-15,0.00,0
3,"РРС ДВ ""Камчатка""",Вилючинск Вилкова КБТ,3,Выдача интернет-заказа,2023-08-08,2023-08-15,1.69,4
4,"РРС ДВ ""Хабаровск""",Облучье ТЦ Статик КБТ,3,Выдача интернет-заказа,2023-08-08,2023-08-15,0.68,1
...,...,...,...,...,...,...,...,...
118,"РРС ДВ ""Сахалин""",Южно-Сахалинск Бумажная Гипер,0,Выдача интернет-заказа,2023-08-08,2023-08-15,2.79,128
119,"РРС ДВ ""Хабаровск""",Хаб ТЦ СВИФ КБТ,0,Выдача интернет-заказа,2023-08-08,2023-08-15,2.96,9
120,"РРС ДВ ""Хабаровск""",Вяземский на Орджоникидзе КБТ,0,Выдача интернет-заказа,2023-08-08,2023-08-15,0.00,0
121,"РРС ДВ ""Владивосток""",Интернет Магазин Артем,0,Выдача интернет-заказа,2023-08-08,2023-08-15,0.00,0


In [146]:
vidacha_iz_rrs

,РРС,Филиал,ТипЗадания,date_start,date_end,ВремяВыполненияПорученияМинут,КоличествоПоручений
0,"РРС ДВ ""Владивосток""",Владивосток ТЦ Первореченский SMART,Выдача интернет-заказа,2023-08-08,2023-08-15,0.78,16
1,"РРС ДВ ""Владивосток""",Находка ТЦ Мега Гипер,Выдача интернет-заказа,2023-08-08,2023-08-15,4.97,107
2,"РРС ДВ ""Сахалин""",Южно-Сахалинск Ленина ТП,Выдача интернет-заказа,2023-08-08,2023-08-15,2.47,92
3,"РРС ДВ ""Владивосток""",Артем ТЦ Конкорд,Выдача интернет-заказа,2023-08-08,2023-08-15,2.03,85
4,"РРС ДВ ""Комсомольск""",Совгавань на Пионерской Гипер,Выдача интернет-заказа,2023-08-08,2023-08-15,2.69,95
...,...,...,...,...,...,...,...
90,"РРС ДВ ""Приморье""",Арсеньев ТЦ Радуга Гипер,Выдача интернет-заказа,2023-08-08,2023-08-15,3.81,287
91,"РРС ДВ ""Владивосток""",Владивосток Снеговая падь МБТ,Выдача интернет-заказа,2023-08-08,2023-08-15,3.97,51
92,"РРС ДВ ""Владивосток""",Владивосток ТЦ Маяк КБТ,Выдача интернет-заказа,2023-08-08,2023-08-15,3.95,89
93,"РРС ДВ ""Владивосток""",Артем Пушкина КБТ,Выдача интернет-заказа,2023-08-08,2023-08-15,3.25,188


In [145]:
df_directors=pd.DataFrame(vidacha_iz_rrs.loc[vidacha_iz_rrs['РРС'] == rrs])

NameError: name 'rrs' is not defined

In [ ]:
df_dir_mail=pd.DataFrame(vidacha_iz_rrs.loc[vidacha_iz_rrs['РРС'] == rrs])

In [ ]:
df_directors.columns = ['РРС', 'Филиал', 'Тип Задания', 'date_start', 'date_end', 'Выполнение поручения в минутах', 'Количество поручений']

In [ ]:
#тестовое отправление самому себе
for rrs in rrs_directors:
    #print(rrs)
    
    #time.sleep(10)
    subject = 'Скорость выполнения выдачи интернет-заказа'
    rrs_body = '<html><body>'
    rrs_body += f'<h1>Скорость выполнения выдачи интернет-заказа в {rrs} за близжайшую неделю (c {date_start} по {date_end}):</h1>\n\n'
    rrs_table = build_table(df_directors.drop(['РРС','date_start', 'date_end'], axis=1), 'orange_dark', font_size='11px', padding= '0px , 0px,  0px , 0px', text_align='center', width_dict=['310px', '310px', '310px', '310px'])
    rrs_body += f"{rrs_table}" 

    rrs_body += '<p><em>Письмо сформировано автоматически, отвечать не требуется</em><br /><em>С уважением, Отдел сопровождения проектов</em></p>'
    rrs_body += '</body></html>'
    rrs_email_text = MIMEText(rrs_body, 'html')
    rrs_email_text['From'] = sender_email
    rrs_email_text['To'] = 'USER_EMAIL'
    rrs_email_text['CC'] = ", ".join(distribution_email)
    rrs_email_text['Subject'] = Header(subject, 'utf-8')
    ## Отправка письма
    time.sleep(3)
    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(login, password)
        
        server.sendmail('USER_EMAIL' ,'USER_EMAIL', rrs_email_text.as_string())
        print(f"Сообщение отправлено на {'USER_EMAIL'}")
        time.sleep(3)
        server.quit()
    except Exception as e:
        print(f"Ошибка с отправкой сообщения на {'USER_EMAIL'}: {str(e)}")

    
df_directors

In [150]:
kluster_rrs

,РРС,Филиал,Итоговый кластер,Тип Задания,date_start,date_end,Выполнение поручения в минутах,Количество поручений
0,"РРС ДВ ""Владивосток""",Владивосток ТЦ Первореченский SMART,3,Выдача интернет-заказа,2023-08-08,2023-08-15,0.78,16
1,"РРС ДВ ""Благовещенск""",Завитинск ТЦ Наш КБТ,3,Выдача интернет-заказа,2023-08-08,2023-08-15,0.00,0
2,"РРС ДВ ""Хабаровск""",Хаб ТЦ Воронежский КБТ,1,Выдача интернет-заказа,2023-08-08,2023-08-15,0.00,0
3,"РРС ДВ ""Камчатка""",Вилючинск Вилкова КБТ,3,Выдача интернет-заказа,2023-08-08,2023-08-15,1.69,4
4,"РРС ДВ ""Хабаровск""",Облучье ТЦ Статик КБТ,3,Выдача интернет-заказа,2023-08-08,2023-08-15,0.68,1
...,...,...,...,...,...,...,...,...
118,"РРС ДВ ""Сахалин""",Южно-Сахалинск Бумажная Гипер,0,Выдача интернет-заказа,2023-08-08,2023-08-15,2.79,128
119,"РРС ДВ ""Хабаровск""",Хаб ТЦ СВИФ КБТ,0,Выдача интернет-заказа,2023-08-08,2023-08-15,2.96,9
120,"РРС ДВ ""Хабаровск""",Вяземский на Орджоникидзе КБТ,0,Выдача интернет-заказа,2023-08-08,2023-08-15,0.00,0
121,"РРС ДВ ""Владивосток""",Интернет Магазин Артем,0,Выдача интернет-заказа,2023-08-08,2023-08-15,0.00,0


In [12]:
email[2]

NameError: name 'email' is not defined

In [13]:
email_list

NameError: name 'email_list' is not defined

In [199]:
len(email_list['Филиал'].unique())

120

In [208]:
df_filial

,Дата,Филиал,Тип Задания,Выполнение поручения в минутах,Количество поручений
4,2023-08-14,Свободный Кручинина КБТ,Выдача интернет-заказа,2.97,14
3,2023-08-13,Свободный Кручинина КБТ,Выдача интернет-заказа,5.24,20
6,2023-08-12,Свободный Кручинина КБТ,Выдача интернет-заказа,1.84,11
5,2023-08-11,Свободный Кручинина КБТ,Выдача интернет-заказа,4.37,10
1,2023-08-10,Свободный Кручинина КБТ,Выдача интернет-заказа,4.19,11
0,2023-08-09,Свободный Кручинина КБТ,Выдача интернет-заказа,3.94,17
2,2023-08-08,Свободный Кручинина КБТ,Выдача интернет-заказа,5.95,21


In [207]:
    df_user=pd.DataFrame(email_list.loc[email_list['email'] == email[2]])
    df_filial=pd.DataFrame(vidacha_iz.loc[vidacha_iz['Филиал'] == list(df_user['Филиал'])[0]])
    df_filial = (df_filial 
                    .reset_index(drop=True)
                    .sort_values (by = ['ДатаВыпПоручения'], ascending = [ False ])
                    .drop(['Филиалguid'], axis=1)             
                )
    df_filial.columns = ['Дата', 'Филиал', 'Тип Задания', 'Выполнение поручения в минутах', 'Количество поручений']
    # Формирование текста письма
    u_f=df_user['Филиал'].to_string(index=False)

In [16]:
#Отправка каждому управляющему магазином письма с данными конкретно по его магазину
empty_filials_name = []
a=0
for el in email:
    #print(el)
    df_user=pd.DataFrame(email_list.loc[email_list['email'] == el])
    df_filial=pd.DataFrame(vidacha_iz.loc[vidacha_iz['Филиал'] == list(df_user['Филиал'])[0]])
    df_filial = (df_filial 
                    .reset_index(drop=True)
                    .sort_values (by = ['ДатаВыпПоручения'], ascending = [ True ])
                    .drop(['Филиалguid'], axis=1) 
                )
    df_filial.columns = ['Дата', 'Филиал', 'Тип Задания', 'Выполнение поручения в минутах', 'Количество поручений']
    # Формирование текста письма
    u_f=df_user['Филиал'].to_string(index=False)
    if df_filial.empty:
        print(f'{u_f} IS EMPTY')
        empty_filials_name.append([df_user['Филиал'].to_string(index=False), df_user['email'].to_string(index=False)])
    else:
        #time.sleep(10)
        subject = f'Скорость выполнения выдачи интернет-заказа в филиале {u_f} {el}'
        body = '<html><body>'
        body += '<h1>Скорость выполнения выдачи интернет-заказа в вашем розничном магазине по дням за близжайшую неделю:</h1>\n\n'
        table = build_table(df_filial, 'orange_dark', font_size='11px', padding= '0px , 0px,  0px , 0px', text_align='center',width_dict=['310px', '310px', '310px', '310px', '310px'])
        body += f"{table}"
        body += '<p><em>Письмо сформировано автоматически, отвечать не требуется</em><br /><em>С уважением, Отдел сопровождения проектов</em></p>'
        body += '</body></html>'
        email_text = MIMEText(body, 'html')
        email_text['From'] = sender_email
        email_text['To'] = el
        #email_text['CC'] = ", ".join(distribution_email)
        email_text['Subject'] = Header(subject, 'utf-8')
        time.sleep(3)
        try:
            server = smtplib.SMTP(smtp_server, smtp_port)
            server.starttls()
            server.login(login, password)
            server.sendmail(el ,sender_email, email_text.as_string())
            print(f"Сообщение отправлено на {el}")
            time.sleep(3)
            server.quit()
        except Exception as e:
            print(f"Ошибка с отправкой сообщения на {sender_email}: {str(e)}")
        a+=1
        if a>=4:
            break

    


NameError: name 'email' is not defined

In [17]:
email_list

NameError: name 'email_list' is not defined

In [ ]:
!pip install openpyxl

In [ ]:
!pip install pendulum

In [ ]:
!pip install psycopg2

In [ ]:
!pip install apache-airflow

In [ ]:
!pip install apache-airflow.providers.telegram

In [ ]:
!pip install pretty_html_table

In [ ]:
!pip install sqlalchemy